## Multiple regression for object-selectivity
This code runs two nested multiple linear regression: one with object-related regressors and the 
other with both object and choice regressors.

R-squared valuese of the two models are compared and if choice regressors improve the model a lot, the neuron is considered to have significant choice information, rather than object identity.


In [8]:
import os
from pathlib import Path
import numpy as np
import pandas as pd

from scipy import stats
from scipy.ndimage import gaussian_filter
from sklearn.linear_model import LinearRegression
from statsmodels.regression.linear_model import OLS
from statsmodels.tools import add_constant

import matplotlib as mpl
import matplotlib.pyplot as plt  

from datetime import date
import time

import random

from joblib import Parallel, delayed

import h5py

In [3]:
# no top and right spines in all plots
mpl.rcParams['axes.spines.right'] = False
mpl.rcParams['axes.spines.top'] = False

In [4]:
mother_path = Path('D:/Multi-modal project/')

### Parameter setting

In [5]:
num_iter = 10

gauss_sigma = 1

# colors for multimodal, vis-only, aud-only conditions
color = ['mediumorchid','cornflowerblue','lightcoral','gray']
color2 = ['cyan','magenta','brown']
linestyle = ['-',':']

today = str(date.today())

### Data preparation

In [61]:
save_path = mother_path /'analysis'/'result'/'3. Multiple regression for object-selectivity'/today
cell_path = mother_path/'analysis'/'result'/'zFR export'/'13-Apr-2022 (5 trials)'

os.makedirs(save_path,exist_ok=True)
f = h5py.File(save_path/f'{today}_multiple_regression_result.hdf5','w')

cell_list = os.listdir(cell_path)

# Data analysis

In [9]:
def groupby_index(df):
    """
    This function is to extract trial indices of each stimulus condition.
    Those indices are used to subsample using another function.
    """
    gb = df.groupby(['Visual', 'Auditory'])

    Boy_M_id = gb.get_group(('Boy', boy_aud)).index.to_numpy()
    Boy_V_id = gb.get_group(('Boy', 'no')).index.to_numpy()
    Boy_A_id = gb.get_group(('no', boy_aud)).index.to_numpy()
    
    Egg_M_id = gb.get_group(('Egg', egg_aud)).index.to_numpy()
    Egg_V_id = gb.get_group(('Egg', 'no')).index.to_numpy()
    Egg_A_id = gb.get_group(('no', egg_aud)).index.to_numpy()
    
    C_L_id = gb.get_group(('Left', 'no')).index.to_numpy()
    C_R_id = gb.get_group(('Right', 'no')).index.to_numpy()

    cond_id = [Boy_M_id, Boy_V_id, Boy_A_id, Egg_M_id, Egg_V_id, Egg_A_id, C_L_id, C_R_id]  
    
    return cond_id

In [58]:
def plot_SDF_beta(df,linewidth,smooth,save,save_format):
    """
    This function plots mean firing rate patterns of each stimulus condition
    and beta coefficients for visual and auditory regressors in multiple linear regression.
    """
    cond = [(df.Type=='Multimodal')&(df.RWD_Loc==boy_goal),
            (df.Type=='Multimodal')&(df.RWD_Loc==egg_goal),
            (df.Type=='Visual')&(df.RWD_Loc==boy_goal),
            (df.Type=='Visual')&(df.RWD_Loc==egg_goal),
            (df.Type=='Auditory')&(df.RWD_Loc==boy_goal),
            (df.Type=='Auditory')&(df.RWD_Loc==egg_goal),
            (df.Type=='Elemental')&(df.RWD_Loc==boy_goal),
            (df.Type=='Elemental')&(df.RWD_Loc==egg_goal)]
    
    cell_full_name = cell_name.strip('.csv')
    
    fr_mean = np.zeros((len(cond),95))
    fr_sem = np.zeros((len(cond),95))
    for i in range(len(cond)):
        fr_mean[i,:] = df[cond[i]].iloc[:,fr_id:fr_id+95].to_numpy().mean(axis=0)
        fr_sem[i,:] = stats.sem(df[cond[i]].iloc[:,fr_id:fr_id+95].to_numpy())
    
    if smooth:
        for i in range(len(cond)):
            fr_mean[i,:] = gaussian_filter(fr_mean[i,:],sigma=gauss_sigma)
            fr_sem[i,:] = gaussian_filter(fr_sem[i,:],sigma=gauss_sigma)
            
    y_max = np.ceil(np.max(fr_mean+fr_sem))
    y_min = np.ceil(np.abs(np.min(fr_mean-fr_sem)))*-1
    
    fig,ax = plt.subplots(3,3,figsize=(15,10))
    plt.suptitle(cell_full_name,fontsize=15);
    x = np.arange(95)*10
    
    for i in range(len(cond)):
        if i%2==0:
            ls = linestyle[0]
        else:
            ls = linestyle[1]            
        ax[0,0].plot(x,fr_mean[i,:],color=color[int(np.floor(i/2))],linewidth=linewidth,linestyle=ls)
        ax[0,0].fill_between(x,fr_mean[i,:]-fr_sem[i,:],fr_mean[i,:]+fr_sem[i,:],color=color[int(np.floor(i/2))],alpha=0.2)
        
        ax[i%2+1,0].plot(x,fr_mean[i,:],color=color[int(np.floor(i/2))],linewidth=linewidth,linestyle=ls)
        ax[i%2+1,0].fill_between(x,fr_mean[i,:]-fr_sem[i,:],fr_mean[i,:]+fr_sem[i,:],color=color[int(np.floor(i/2))],alpha=0.2)
        
        if i<6:
            ax[int(np.floor(i/2)),1].plot(x,fr_mean[i,:],color=color[int(np.floor(i/2))],linewidth=linewidth,linestyle=ls)
            ax[int(np.floor(i/2)),1].fill_between(x,fr_mean[i,:]-fr_sem[i,:],fr_mean[i,:]+fr_sem[i,:],color=color[int(np.floor(i/2))],alpha=0.2)
    
    # control stimulus SDF
    for i in range(3):
        ax[i,1].plot(x,fr_mean[6,:],color=color[3])
        ax[i,1].plot(x,fr_mean[7,:],color=color[3],linestyle=':')
            
    for i in range(6):
        ax[i%3,int(np.floor(i/3))].set_yticks(np.arange(y_min,y_max+0.1,1))
        ax[i%3,int(np.floor(i/3))].set_ylabel('z-scored FR',fontsize=13)
        ax[i%3,int(np.floor(i/3))].set_xlabel('Time (ms)',fontsize=13)  
        ax[i%3,int(np.floor(i/3))].set_xticks([0,400,950])
        ax[i%3,int(np.floor(i/3))].set_xlim([0,950])    
    
    # beta coefficient plot
    t2 = ['Visual term','Auditory term','Interaction term']
    c2 = ['blue','red','green']
    for i in range(3):
        ax[i,2].plot(x,beta_mean[:,i],color=c2[i],linestyle='-',linewidth=linewidth)
        ax[i,2].plot(x,beta_mean[:,i+3],color=c2[i],linestyle=':',linewidth=linewidth)
        
        ax_r = ax[i,2].twinx()
        ax_r.plot(x,r_diff,color='lightgray',linewidth=1)
        ax_r.set_ylabel('R-squared difference')   
        #ax_r.set_yticks(np.arange(0,0.21,0.04))       
            
        ax[i,2].set_title(t2[i],fontsize=13)
        ax[i,2].set_yticks(np.arange(-1,1.1,0.5))    
        ax[i,2].set_ylabel('Beta coefficient',fontsize=13)
        ax[i,2].set_xlabel('Time (ms)',fontsize=13)    
        ax[i,2].set_xticks([0,400,950])
        ax[i,2].set_xlim([0,950])
    
    # mark time bins with significant difference in beta coefficients
    ax[0,2].scatter(vis_sig_bin[0]*10,np.tile(0.8,(len(vis_sig_bin[0]),1)),marker='*',color='black')
    ax[1,2].scatter(aud_sig_bin[0]*10,np.tile(0.8,(len(aud_sig_bin[0]),1)),marker='*',color='black')
    plt.tight_layout()
    
    if save:          
        fig_path = save_path/region
        if os.path.exists(fig_path) is False:
            os.makedirs(fig_path)    
        if save_format=='png':
            plt.savefig(fig_path/f'{cell_full_name}.png',dpi=100,facecolor='white')
        elif save_format=='svg':
            plt.savefig(fig_path/f'{cell_full_name}.svg')
        plt.close()

In [44]:
def subsample_and_MLR(df,num_trials):
    """
    This function subsamples trials from each stimulus condition
    and calculate beta coefficients and r-squared values of multiple linear regression.
    """
    subsample = np.array([])
    # subsample 5 trials from each stimulus condition
    for c in cond_id:
        subsample = np.append(subsample,np.random.choice(c,num_trials,replace=True))        
        subsample = np.asarray(subsample,dtype=int)
    
    # set regressors for the reduced and full model
    x1 = df.loc[subsample,['Boy-V','Boy-A','Boy-int','Egg-V','Egg-A','Egg-int','constant']]
    x2 = df.loc[subsample,['Boy-V','Boy-A','Boy-int','Egg-V','Egg-A','Egg-int','Choice','constant']]    
            
    # shuffle trial conditions to get null distribution
    state = np.random.get_state()
    x1_shuffle = np.random.permutation(x1)
    np.random.set_state(state)
    x2_shuffle = np.random.permutation(x2)
    
    beta_coef, beta_coef_choice, beta_coef_shuffle, beta_coef_choice_shuffle = [],[],[],[]
    rsquared, rsquared_choice, rsquared_shuffle, rsquared_choice_shuffle = [],[],[],[]
    BIC, BIC_choice, BIC_shuffle, BIC_choice_shuffle = [],[],[],[]
    AIC, AIC_choice, AIC_shuffle, AIC_choice_shuffle = [],[],[],[]
    
    # calculate beta coefficients and r-squared values in each time bin
    for t in range(95):
        y = fr[subsample,t]
            
        lr1 = OLS(y,add_constant(x1)).fit()
        lr2 = OLS(y,add_constant(x2)).fit()
    
        lr1_shuffle = OLS(y,add_constant(x1_shuffle)).fit()
        lr2_shuffle = OLS(y,add_constant(x2_shuffle)).fit()
            
        beta_coef.append(lr1.params[0:-1])
        beta_coef_choice.append(lr2.params[0:-1])
        beta_coef_shuffle.append(lr1_shuffle.params[0:-1])
        beta_coef_choice_shuffle.append(lr2_shuffle.params[0:-1])
        
        rsquared.append(lr1.rsquared)
        rsquared_choice.append(lr2.rsquared)
        rsquared_shuffle.append(lr1_shuffle.rsquared)
        rsquared_choice_shuffle.append(lr2_shuffle.rsquared)
        
        BIC.append(lr1.bic)
        BIC_choice.append(lr2.bic)
        BIC_shuffle.append(lr1_shuffle.bic)
        BIC_choice_shuffle.append(lr2_shuffle.bic)
        
        AIC.append(lr1.aic)
        AIC_choice.append(lr2.aic)
        AIC_shuffle.append(lr1_shuffle.aic)
        AIC_choice_shuffle.append(lr2_shuffle.aic)
        
    return beta_coef, beta_coef_choice, beta_coef_shuffle, beta_coef_choice_shuffle,\
            rsquared, rsquared_choice, rsquared_shuffle, rsquared_choice_shuffle,\
            BIC, BIC_choice, BIC_shuffle, BIC_choice_shuffle,\
            AIC, AIC_choice, AIC_shuffle, AIC_choice_shuffle

In [50]:
def save_result(f):
    """
    This function saves data into HDF5 format.
    """
    cell_group = f.create_group(str(cell_id))
    beta_group = f.create_group(f'{cell_id}/beta_coef')
    rs_group = f.create_group(f'{cell_id}/rsquared')
    BIC_group = f.create_group(f'{cell_id}/BIC')
    AIC_group = f.create_group(f'{cell_id}/AIC')
    
    beta_group.create_dataset('basic',data=beta_coef)
    beta_group.create_dataset('extended',data=beta_coef_choice)
    beta_group.create_dataset('basic_shuffle',data=beta_coef_shuffle)
    beta_group.create_dataset('extended_shuffle',data=beta_coef_choice_shuffle)
    
    rs_group.create_dataset('basic',data=rsquare)
    rs_group.create_dataset('extended',data=rsquare_choice)
    rs_group.create_dataset('basic_shuffle',data=rsquare_shuffle)
    rs_group.create_dataset('extended_shuffle',data=rsquare_choice_shuffle)
    
    BIC_group.create_dataset('basic',data=BIC)
    BIC_group.create_dataset('extended',data=BIC_choice)
    BIC_group.create_dataset('basic_shuffle',data=BIC_shuffle)
    BIC_group.create_dataset('extended_shuffle',data=BIC_choice_shuffle)    
    
    AIC_group.create_dataset('basic',data=BIC)
    AIC_group.create_dataset('extended',data=BIC_choice)
    AIC_group.create_dataset('basic_shuffle',data=BIC_shuffle)
    AIC_group.create_dataset('extended_shuffle',data=BIC_choice_shuffle)           
    
    cell_group.attrs['Rat'] = rat_id
    cell_group.attrs['Region'] = region
    cell_group.attrs['Session'] = session_id
    
    cell_group.attrs['Visual'] = int(len(vis_sig_bin[0])!=0)
    cell_group.attrs['Auditory'] = int(len(aud_sig_bin[0])!=0)

In [62]:
%%time
for cell_run,cell_name in enumerate(cell_list):
    loop_start = time.time()
    # get information about the cell
    cell_info = cell_name.split('-')
    cell_id, rat_id, session_id, region = int(cell_info[0]), cell_info[1], cell_info[2], cell_info[5]
    
    if (rat_id=='654')&(session_id=='4'):
        continue
    
    # load cell data
    df = pd.read_csv(cell_path/cell_name)
    df.drop(df[df.Correctness==0].index,inplace=True)
    df.reset_index(inplace=True,drop=True)
    df[['Visual','Auditory']] = df[['Visual','Auditory']].fillna('no')
    
    boy_goal = df.loc[df['Visual']=='Boy','RWD_Loc'].values[0]
    boy_aud = df.loc[df['RWD_Loc']==boy_goal,'Auditory'].values[0]
    
    egg_goal = df.loc[df['Visual']=='Egg','RWD_Loc'].values[0]
    egg_aud = df.loc[df['RWD_Loc']==egg_goal,'Auditory'].values[0]  
    
    df['Boy-V'] = (df['Visual'] == 'Boy').astype(int)
    df['Boy-A'] = (df['Auditory'] == boy_aud).astype(int)
    df['Egg-V'] = (df['Visual'] == 'Egg').astype(int)
    df['Egg-A'] = (df['Auditory'] == egg_aud).astype(int)
    
    df['Boy-int'] = df['Boy-V']*df['Boy-A']
    df['Egg-int'] = df['Egg-V']*df['Egg-A']
    
    df['Choice'] = (df['RWD_Loc']==boy_goal).astype(int) 
    df['constant'] = 1
    
    fr_id = df.columns.get_loc('Var10')  # get the index of the first firing rate column
    fr = df.iloc[:,fr_id:fr_id+95].to_numpy()    # get firing rate data into array
    
    # get trial indices of each stimulus condition
    cond_id = groupby_index(df)    
        
    # subsample and run multiple linear regression for n times
    results = Parallel(n_jobs=-1)(delayed(subsample_and_MLR)(df,5) for i in range(num_iter))
    
    beta_coef = np.array([r[0] for r in results])
    beta_coef_choice = np.array([r[1] for r in results])
    beta_coef_shuffle = np.array([r[2] for r in results])
    beta_coef_choice_shuffle = np.array([r[3] for r in results])
    
    rsquare = np.array([r[4] for r in results])
    rsquare_choice = np.array([r[5] for r in results])
    rsquare_shuffle = np.array([r[6] for r in results])
    rsquare_choice_shuffle = np.array([r[7] for r in results])
    
    BIC = np.array([r[8] for r in results])
    BIC_choice = np.array([r[9] for r in results])
    BIC_shuffle = np.array([r[10] for r in results])
    BIC_choice_shuffle = np.array([r[11] for r in results])
    
    AIC = np.array([r[12] for r in results])
    AIC_choice = np.array([r[13] for r in results])
    AIC_shuffle = np.array([r[14] for r in results])
    AIC_choice_shuffle = np.array([r[15] for r in results])    

    # get mean beta coefficient from subsampled population
    beta_mean = np.mean(beta_coef,axis=0)
    beta_std = np.std(beta_coef,axis=0)    
    
    # r-squared difference between full model and reduced model to quantify choice-related signals
    rsquare[np.where(rsquare==np.inf*-1)] = np.nan
    rsquare_choice[np.where(rsquare_choice==np.inf*-1)] = np.nan
    r_diff = np.nanmean(rsquare_choice,axis=0)-np.nanmean(rsquare,axis=0)
    
    # find bins with significant differences in beta coefficients between two objects
    vis_diff = beta_coef_shuffle[:,:,0] - beta_coef_shuffle[:,:,3]
    aud_diff = beta_coef_shuffle[:,:,1] - beta_coef_shuffle[:,:,4]
    vis_crit = np.percentile(np.ravel(vis_diff), [2.5, 97.5])
    aud_crit = np.percentile(aud_diff, [2.5, 97.5])        
    vis_sig_bin = np.where(((beta_mean[:,0]-beta_mean[:,3])>vis_crit[1])|((beta_mean[:,0]-beta_mean[:,3])<vis_crit[0]))
    aud_sig_bin = np.where(((beta_mean[:,1]-beta_mean[:,4])>aud_crit[1])|((beta_mean[:,1]-beta_mean[:,4])<aud_crit[0]))        

    plot_SDF_beta(df,2,1,1,'png')
    
    # save results into HDF5 format
    save_result(f)
            
    loop_end = time.time()
    loop_time = divmod(loop_end-loop_start,60)
    print(cell_name.strip('.csv'), f'////// {cell_run+1}/{len(cell_list)} completed  //////  {int(loop_time[0])} min {loop_time[1]:.2f} sec')

0003-600-1-1-Crossmodal-TeV-deep-(-7.32 mm)-TT4.1 ////// 1/888 completed  //////  0 min 3.81 sec
0004-600-1-1-Crossmodal-TeV-deep-(-7.32 mm)-TT4.2 ////// 2/888 completed  //////  0 min 4.17 sec
0005-600-1-1-Crossmodal-TeV-deep-(-7.32 mm)-TT4.3 ////// 3/888 completed  //////  0 min 4.26 sec
0006-600-1-1-Crossmodal-PER-superficial-(-7.2 mm)-TT5.1 ////// 4/888 completed  //////  0 min 3.75 sec
0007-600-1-1-Crossmodal-PER-superficial-(-7.2 mm)-TT5.2 ////// 5/888 completed  //////  0 min 4.46 sec
0008-600-1-1-Crossmodal-PER-superficial-(-7.2 mm)-TT5.3 ////// 6/888 completed  //////  0 min 4.28 sec


<timed exec>:63: RuntimeWarning: invalid value encountered in subtract


0010-600-1-1-Crossmodal-TeV-deep-(-6.96 mm)-TT6.2 ////// 7/888 completed  //////  0 min 5.29 sec


<timed exec>:63: RuntimeWarning: invalid value encountered in subtract


0011-600-1-1-Crossmodal-PER-superficial-(-6.48 mm)-TT7.1 ////// 8/888 completed  //////  0 min 5.51 sec
0013-600-1-1-Crossmodal-PER-deep-(-6.48 mm)-TT8.2 ////// 9/888 completed  //////  0 min 5.92 sec


...

<timed exec>:63: RuntimeWarning: invalid value encountered in subtract


0041-600-2-2-Crossmodal-PER-superficial-(-6.36 mm)-TT15.1 ////// 34/888 completed  //////  0 min 3.47 sec
0042-600-2-2-Crossmodal-TeV-superficial-(-6.96 mm)-TT17.1 ////// 35/888 completed  //////  0 min 3.69 sec
0043-600-2-2-Crossmodal-TeV-superficial-(-6.96 mm)-TT17.2 ////// 36/888 completed  //////  0 min 4.20 sec
0044-600-2-2-Crossmodal-TeV-superficial-(-6.96 mm)-TT17.3 ////// 37/888 completed  //////  0 min 3.55 sec
0045-600-2-2-Crossmodal-TeV-deep-(-6.96 mm)-TT19.1 ////// 38/888 completed  //////  0 min 3.56 sec
0046-600-2-2-Crossmodal-POR-deep-(-7.56 mm)-TT24.1 ////// 39/888 completed  //////  0 min 3.67 sec
0047-600-2-2-Crossmodal-POR-deep-(-7.56 mm)-TT24.2 ////// 40/888 completed  //////  0 min 4.69 sec
0048-600-2-2-Crossmodal-POR-deep-(-7.56 mm)-TT24.3 ////// 41/888 completed  //////  0 min 3.71 sec
0049-600-3-3-Crossmodal-PER-superficial-(-7.2 mm)-TT5.1 ////// 42/888 completed  //////  0 min 3.78 sec
0050-600-3-3-Crossmodal-PER-superficial-(-7.2 mm)-TT5.2 ////// 43/888 comple

<timed exec>:63: RuntimeWarning: invalid value encountered in subtract


0063-600-4-4-Crossmodal-PER-deep-(-6.96 mm)-TT6.1 ////// 54/888 completed  //////  0 min 5.21 sec
0065-600-4-4-Crossmodal-PER-superficial-(-6.48 mm)-TT7.1 ////// 55/888 completed  //////  0 min 3.74 sec
0066-600-4-4-Crossmodal-PER-superficial-(-6.48 mm)-TT7.2 ////// 56/888 completed  //////  0 min 4.27 sec


KeyboardInterrupt: 

In [60]:
f.close()
print('END')

END
